In [1]:
from sys import path
path.append("../")

# ARDC Training: Python Notebooks
Task-F: Data Export
<br><br>
The code in this notebook subsets a data cube, selects a specific set of variables, creates a new XARRAY, and then outputs that data into a GeoTIFF file and CSV file. This output file can be used in other software programs (e.g. QGIS, ArcGIS, EXCEL) for more specific analyses. We will keep the region small so that we can control file sizes. 

> ### Import the Datacube Configuration

In [1]:
import datacube
dc = datacube.Datacube(app = 'my_app', config = '/home/localuser/.datacube.conf')

# Supress Warning 
import warnings
warnings.filterwarnings('ignore')

>### Browse the available Data Cubes on the storage platform    
> You might want to learn more about what data is stored and how it is stored.


In [2]:
list_of_products = dc.list_products()
netCDF_products = list_of_products[list_of_products['format'] == 'NetCDF']
netCDF_products

,name,description,lon,platform,product_type,instrument,lat,format,time,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,
61,alos2_jjfast_scansar_tile_colombia,ALOS2 PALSAR JJFAST tile in DN format processe...,None,ALOS_2,tile,PALSAR,None,NetCDF,None,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
65,alos2_palsar2_scansar_caqueta,ALOS2 PALSAR JJFAST tile in DN format processe...,None,ALOS_2,tile,PALSAR_2,None,NetCDF,None,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
50,alos2_palsar_colombia,ALOS2 PALSAR tile in DN format processed for t...,None,ALOS_2,gamma0,PALSAR,None,NetCDF,None,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
51,alos2_palsar_kenya,ALOS2 PALSAR tile in DN format processed for t...,None,ALOS_2,gamma0,PALSAR,None,NetCDF,None,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
52,alos2_palsar_vietnam,ALOS2 PALSAR tile in DN format processed for t...,None,ALOS_2,gamma0,PALSAR,None,NetCDF,None,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
53,alos_palsar_colombia,ALOS PALSAR tile in DN format processed for th...,None,ALOS,gamma0,PALSAR,None,NetCDF,None,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
54,alos_palsar_kenya,ALOS PALSAR tile in DN format processed for th...,None,ALOS,gamma0,PALSAR,None,NetCDF,None,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
55,alos_palsar_vietnam,ALOS PALSAR tile in DN format processed for th...,None,ALOS,gamma0,PALSAR,None,NetCDF,None,EPSG:4326,"[-0.0002666666667, 0.0002666666667]","[0.5333333334, 0.5333333334]","(latitude, longitude)"
14,gpm_imerg_gis_daily_global,Global NetCDF GPM IMERG GIS data,None,GPM,daily,GPM,None,NetCDF,None,EPSG:4326,"[-0.1, 0.1]","[90, 180]","(latitude, longitude)"


>### Pick a product  
>Use the platform and product names from the previous block to select a Data Cube.  

In [3]:
import utils.data_cube_utilities.data_access_api as dc_api  
api = dc_api.DataAccessApi(config = '/home/localuser/.datacube.conf')

# Change the data platform and data cube here
# This data export notebook will only work for Landsat-7 datasets

platform = "LANDSAT_7"

# product = "ls7_ledaps_ghana"
# product = "ls7_ledaps_kenya"
# product = "ls7_ledaps_senegal"
# product = "ls7_ledaps_sierra_leone"
# product = "ls7_ledaps_tanzania"
product = "ls7_ledaps_vietnam"

# Get Coordinates
coordinates = api.get_full_dataset_extent(platform = platform, product = product)

# Print out Latitude, Longitude and Time Extents
latitude_extents = (min(coordinates['latitude'].values),max(coordinates['latitude'].values))
print( latitude_extents )
longitude_extents = (min(coordinates['longitude'].values),max(coordinates['longitude'].values))
print( longitude_extents )
time_extents = (min(coordinates['time'].values),max(coordinates['time'].values))
print( time_extents )

(9.1764253745784181, 13.964805165051667)
(102.40430421277932, 108.93092407802477)
(numpy.datetime64('1999-09-08T03:13:19.000000000'), numpy.datetime64('2016-12-29T03:10:00.000000000'))


# Visualize Data Cube Region

In [4]:
## The code below renders a map that can be used to orient yourself with the region.
from utils.data_cube_utilities.dc_display_map import display_map
display_map(latitude = latitude_extents, longitude = longitude_extents)

> ### Pick a smaller analysis region and display that region
Try to keep your region to less than 0.2-deg x 0.2-deg for rapid processing. You can click on the map above to find the Lat-Lon coordinates of any location. Pick a time window of 1 year to keep the file small.

In [5]:
## Vietnam - Central Lam Dong Province ##
longitude_extents = (105.2, 105.5)
latitude_extents  = (11.25, 11.55)

time_extents = ('2010-01-01', '2011-01-01')
print ( time_extents )

('2010-01-01', '2011-01-01')


In [6]:
from utils.data_cube_utilities.dc_display_map import display_map
display_map(latitude = latitude_extents, longitude = longitude_extents)

In [7]:
landsat_dataset = dc.load(latitude = latitude_extents,
                          longitude = longitude_extents,
                          platform = platform,
                          time = time_extents,
                          product = product,
                          measurements = ['red', 'green', 'blue', 'nir', 'swir1', 'swir2', 'pixel_qa']) 

In [8]:
landsat_dataset # this is a printout of the first few values of each parameter in the XARRAY

<xarray.Dataset>
Dimensions:    (latitude: 1115, longitude: 1114, time: 10)
Coordinates:
  * time       (time) datetime64[ns] 2010-01-09T03:11:26 2010-02-26T03:12:02 ...
  * latitude   (latitude) float64 11.55 11.55 11.55 11.55 11.55 11.55 11.55 ...
  * longitude  (longitude) float64 105.2 105.2 105.2 105.2 105.2 105.2 105.2 ...
Data variables:
    red        (time, latitude, longitude) int16 757 881 922 922 921 1045 ...
    green      (time, latitude, longitude) int16 806 828 874 851 941 986 ...
    blue       (time, latitude, longitude) int16 601 600 621 642 597 706 860 ...
    nir        (time, latitude, longitude) int16 2038 1852 1620 1574 1714 ...
    swir1      (time, latitude, longitude) int16 1136 1001 565 483 729 1219 ...
    swir2      (time, latitude, longitude) int16 529 500 327 240 442 616 414 ...
    pixel_qa   (time, latitude, longitude) int32 66 66 66 66 66 66 66 66 66 ...
Attributes:
    crs:      EPSG:4326

# Create Several Common Application Products

>## Unpack pixel_qa
This is the Landsat-7 pixel quality data that is used to screen for clouds, shadows, snow, etc. These values can be quite valuable when doing an analysis in a GIS tool, but you will not need all of them.

In [9]:
import xarray as xr  
import numpy as np

def ls7_unpack_qa( data_array , cover_type):  
    
    land_cover_endcoding = dict( fill         =[1] ,
                                 clear        =[322, 386, 834, 898, 1346],
                                 water        =[324, 388, 836, 900, 1348],
                                 shadow       =[328, 392, 840, 904, 1350],
                                 snow         =[336, 368, 400, 432, 848, 880, 812, 944, 1352],
                                 cloud        =[352, 368, 416, 432, 848, 880, 912, 944, 1352],
                                 low_conf_cl  =[322, 324, 328, 336, 352, 368, 834, 836, 840, 848, 864, 880],
                                 med_conf_cl  =[386, 388, 392, 400, 416, 432, 898, 900, 904, 928, 944],
                                 high_conf_cl =[480, 992],
                                 low_conf_cir =[322, 324, 328, 336, 352, 368, 386, 388, 392, 400, 416, 432, 480],
                                 high_conf_cir=[834, 836, 840, 848, 864, 880, 898, 900, 904, 912, 928, 944], 
                                 terrain_occ  =[1346,1348, 1350, 1352]
                               )
    boolean_mask = np.isin(data_array, land_cover_endcoding[cover_type]) 
    return xr.DataArray(boolean_mask.astype(np.int8),
                        coords = data_array.coords,
                        dims = data_array.dims,
                        name = cover_type + "_mask",
                        attrs = data_array.attrs)  

In [10]:
clear_xarray  = ls7_unpack_qa(landsat_dataset.pixel_qa, "clear")  
water_xarray  = ls7_unpack_qa(landsat_dataset.pixel_qa, "water")
shadow_xarray = ls7_unpack_qa(landsat_dataset.pixel_qa, "shadow") 
cloud_xarray = ls7_unpack_qa(landsat_dataset.pixel_qa, "cloud") 

In [11]:
clean_xarray = xr.ufuncs.logical_or(clear_xarray , water_xarray).astype(np.int8).rename("clean_mask")

clean_mask = np.logical_or(clear_xarray.values.astype(bool),
                           water_xarray.values.astype(bool)) 

> ##  Spectral Indices
Below are a number of common spectral indices. 

In [12]:
def NDVI(dataset):
    return ((dataset.nir - dataset.red)/(dataset.nir + dataset.red)).rename("NDVI")

In [13]:
def NDWI(dataset):
    return ((dataset.green - dataset.nir)/(dataset.green + dataset.nir)).rename("NDWI")

In [14]:
def NDBI(dataset):
        return ((dataset.swir2 - dataset.nir)/(dataset.swir2 + dataset.nir)).rename("NDBI")

In [15]:
def EVI(dataset, c1 = None, c2 = None, L = None):
        return ((dataset.nir - dataset.red)/((dataset.nir  + (c1 * dataset.red) - (c2 *dataset.blue) + L))).rename("EVI")

In [16]:
ndbi_xarray = NDBI(landsat_dataset)  # Urbanization - Reds
ndvi_xarray = NDVI(landsat_dataset)  # Dense Vegetation - Greens
ndwi_xarray = NDWI(landsat_dataset)  # High Concentrations of Water - Blues  
evi_xarray = EVI(landsat_dataset, c1 = 6, c2 = 7.5, L = 1 ) # Enhanced Vegetation Index

>## TSM
This is the Total Suspended Matter (TSM) index which measures the quality of water using a simple equation with one of Landsat bands. For the analysis below we will use the water pixels from the Landsat "pixel_qa" so that the code run faster than using the WOFS water analysis. 

In [17]:
from utils.data_cube_utilities.dc_water_quality import tsm

tsm_xarray = tsm(landsat_dataset, clean_mask = water_xarray.values.astype(bool) ).tsm

# Combine Everything  

In [18]:
combined_dataset = xr.merge([landsat_dataset,
          clean_xarray,
          clear_xarray,
          water_xarray,
          shadow_xarray,
          cloud_xarray,                  
          evi_xarray,
          ndbi_xarray,
          ndvi_xarray,
          ndwi_xarray,
          tsm_xarray])

# Copy original crs to merged dataset 
combined_dataset = combined_dataset.assign_attrs(landsat_dataset.attrs)

combined_dataset  # this is a printout of the first few values of each parameter in the XARRAY

<xarray.Dataset>
Dimensions:      (latitude: 1115, longitude: 1114, time: 10)
Coordinates:
  * time         (time) datetime64[ns] 2010-01-09T03:11:26 ...
  * latitude     (latitude) float64 11.55 11.55 11.55 11.55 11.55 11.55 ...
  * longitude    (longitude) float64 105.2 105.2 105.2 105.2 105.2 105.2 ...
Data variables:
    red          (time, latitude, longitude) int16 757 881 922 922 921 1045 ...
    green        (time, latitude, longitude) int16 806 828 874 851 941 986 ...
    blue         (time, latitude, longitude) int16 601 600 621 642 597 706 ...
    nir          (time, latitude, longitude) int16 2038 1852 1620 1574 1714 ...
    swir1        (time, latitude, longitude) int16 1136 1001 565 483 729 ...
    swir2        (time, latitude, longitude) int16 529 500 327 240 442 616 ...
    pixel_qa     (time, latitude, longitude) int32 66 66 66 66 66 66 66 66 ...
    clean_mask   (time, latitude, longitude) int8 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
    clear_mask   (time, latitude, longitude)

## Export CSV
This section will be used to create a CSV export file for a given pixel. You will identify the pixel by selecting a specific Lat-Lon position and then the code will find the closest pixel to that point (nearest neighbor). Use the map at the top of this notebook to view your region and pick a Lat-Lon location. You can find an exact location by clicking on the map. The CSV file will contain the time series data for each XARRAY parameter.

In [19]:
# Lat and Lon coordinates extracted from the map above 
pixel_lat = 11.3972
pixel_lon = 105.3528

In [20]:
pixel = combined_dataset.sel(latitude  = pixel_lat,
                             longitude = pixel_lon,
                              method = 'nearest') # nearest neighbor selection  

In [21]:
import xarray as xr
import csv

def ts_pixel_to_csv(pixel: xr.Dataset,
                    csv_file_name: str):
    def __yield_from_time_axis(px):
        for i in range(len(px.time)):
            yield px.isel(time = i)
    def __format_time(t):
        return t
    
    with open(csv_file_name,'w') as out:
        csv_out=csv.writer(out)
        column_names = ['time'] + list(pixel.data_vars)
        csv_out.writerow(column_names)

        for row in __yield_from_time_axis(pixel):
            csv_out.writerow([__format_time(row.time.values)] + [row[key].values for key in list(pixel.data_vars)])

In [22]:
csv_name = 'test.csv'

In [23]:
ts_pixel_to_csv(pixel, csv_name)

## Export GeoTIFF
This section will be used to create a GeoTIFF export.

----  
File formatting  

In [24]:
import time
def time_to_string(t):
    return time.strftime("%Y_%m_%d_%H_%M_%S", time.gmtime(t.astype(int)/1000000000))

----  
This function can be used to write a single time slice from an xarray to geotiff

In [25]:
from utils.data_cube_utilities import dc_utilities
def export_slice_to_geotiff(ds, path):
    dc_utilities.write_geotiff_from_xr(path,
                                        ds.astype(np.float32),
                                        list(combined_dataset.data_vars.keys()),
                                        crs="EPSG:4326")

----  
For each time slice in a dataset we call `export_slice_to_geotif`  

In [26]:
def export_xarray_to_geotiff(ds, path):
    for t in ds.time:
        time_slice_xarray = ds.sel(time = t)
        export_slice_to_geotiff(time_slice_xarray,
                                path + "_" + time_to_string(t) + ".tif")

## Start Export
This is where we will start the GeoTIFF export and review the final product. The lines after this text box have been "commented out" so that they can be run at the end, after you have completed the creation of the XARRAY above and reviewed the data. 

In [27]:
# export_xarray_to_geotiff(combined_dataset, "geotiffs/landsat7")

----  
Check to see what files exist in `geotiffs`

In [28]:
# !ls -lah geotiffs/*.tif